In [1]:
#  %%bash
# python -m pip install --upgrade pip
# pip install jupyterlab
# pip install dataclasses
# pip install pandas
# pip install pyarrow
# pip install openpyxl
# pip install requests
!pip install transformers
!pip install torch
!pip install PyPDF2
!pip install faiss-cpu # or faiss-gpu for GPU support
!pip install numpy
!pip install tiktoken
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 19.5 MB/s eta 0:00:00


In [2]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.3 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import PyPDF2
from langchain.text_splitter import TokenTextSplitter
import faiss
import numpy as np


# Initialize tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')

#Mean Pooling - Take attention mask into account for correct averaging

def mean_pooling(model_output, attention_mask):
  token_embeddings = model_output[0] #First element of model_output contains all token embeddings
  input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
  return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def generate_embedding(text):
  # Tokenize input text
  encoded_input = tokenizer(text, padding=True, truncation=True,return_tensors='pt')
  # Compute token embeddings with model
  with torch.no_grad():
    model_output = model(**encoded_input)
  # Perform mean pooling
  sentence_embedding = mean_pooling(model_output,encoded_input['attention_mask'])
  # Convert to numpy for FAISS compatibility and ensure it's 2D
  return sentence_embedding.cpu().numpy().reshape(1, -1)


# Your existing functions like read_pdf, split_into_chunks, save_embeddings,load_faiss_index, search_faiss_index, get_text_for_indices
# Adjusted function to generate embeddings for chunks of text
def generate_embeddings_for_chunks(text_chunks):
  embeddings = np.vstack([generate_embedding(chunk) for chunk in text_chunks])
  return embeddings

# Adjusted function to create and save FAISS index
def save_embeddings(embeddings, file_name="vecstore.faiss"):
  dimension = embeddings.shape[1]
  index = faiss.IndexFlatL2(dimension)
  index.add(embeddings)
  faiss.write_index(index, file_name)

# Load FAISS index
def load_faiss_index(file_name='vecstore.faiss'):
  return faiss.read_index(file_name)

# Search in FAISS index
def search_faiss_index(index, query_embedding, top_n=4):
  # Ensure query_embedding is a 2D array with shape (1, embedding_size)
  query_embedding_2d = query_embedding.reshape(1, -1)
  D, I = index.search(query_embedding_2d, top_n)
  return I[0]

def read_pdf(file_path):
  """Reads a PDF file and returns its text content."""
  with open(file_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    text = ''
    for page_num in range(len(reader.pages)):
      text += reader.pages[page_num].extract_text()
  return text

def split_into_chunks(text):
  """Splits the text into chunks of specific token length and returns a list of those chunks"""
  text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=0)
  texts = text_splitter.split_text(text)
  return texts

def get_text_for_indices(indices, text_chunks):
  """Returns the text chunks for the given indices."""
  return [text_chunks[i] for i in indices]

# Example usage remains the same
pdf_path = '/content/DIKESH RAY.pdf' #␣Specify your PDF path
text = read_pdf(pdf_path)
text_chunks = split_into_chunks(text)
embeddings = generate_embeddings_for_chunks(text_chunks)
save_embeddings(embeddings, "vecstore.faiss")
vecstore = load_faiss_index("vecstore.faiss")

# For querying
# vecstore = load_faiss_index("vecstore.faiss")
# query_text = "You have been given a resume , What is the name of Applicant in the resume in one word ?"
# query_embedding = generate_embedding(query_text)
# top_indices = search_faiss_index(vecstore, query_embedding)
# print("Top similar documents indices:", top_indices)
# # After you perform the search
# top_text_chunks = get_text_for_indices(top_indices, text_chunks)
# print("Top similar documents text chunks:")
# for chunk in top_text_chunks:
# print(chunk)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
# prompt = {
#     "id": "Randomly Create unique value for ID for this resume?   [Only provide single integer]",
#     "fname": "What is the first name present in the resume?  [Only provide single string]",
#     "lastName": "What is the last name present in the resume? [Only provide single string]",
#     "dob": "What is the date of birth present in the resume? [Only provide single string]",
#     "age": "What is the age (Current year - Date of Birth year) present in the resume? [Only provide single integer]",
#     "email": "What is the email present in the resume? [Only provide single string]",
#     "address": "What is the address present in the resume? [Only provide string]",
#     "mobile": "What is the mobile number present in the resume? [Only provide single long integer]",
#     "LinkedIn": "What is the LinkedIn profile present in the resume? [Only provide single link]",
#     "orcidId": "What is the ORCID ID for the research papers present in the resume? [Only provide single id]",
#     "totalIndustryExperience": "What is the total industry years of experience present in the resume. [Sum of all the years of experience and provide integer value+ 'years'] ",
#     "summaryDetails": "What is the summary present in the resume? [Only provide 2 lines string]",
#     "education_id": "Randomly Create education ID as primary key for my dataset, provide a unique value for this resume?  [Only provide single integer]",
#     "degree": "What are the degrees(10th Degree, 12th Degree, Undergrad Degree, Postgrad Degree) present in the resume? Answer in points and keep it precise. [Only provide Degree names]" ,
#     "course": "What are the courses taken in the degrees present in the resume? Answer in points and keep it precise. [Only provide string separated by ',']",
#     "marks": "What are the CGPA (Undergrad Degree, Postgrad Degree) present in the resume? Answer in points and keep it precise. [Only provide integer]",
#     "yearOfPass": "What are the years of passing (for 10th Degree, 12th Degree, Undergrad Degree, Postgrad Degree) present in the resume? Answer in points and keep it precise. [Only provide integer]",
#     "boardsName": "What are the board's names (10th Degree, 12th Degree) present in the resume? Answer in points and keep it precise. [Only provide string]",
#     "percentage": "What are the percentages (10th Degree, 12th Degree) present in the resume? Answer in points  [provide integer value + '% in 10th and % in 12th'] ",
#     "Work Experience": "What is/are the work experience present in resume?  Answer in points and keep it precise. [Only provide single 2 lines string]",
#     "employement_id": "Randomly Create an employment ID as primary key for my dataset, provide a unique value for this resume? [Only provide single integer]",
#     'nameOfEmployer': "What are the names of the employers present in the resume for the particular job positions?  Answer in points and keep it precise. [Only provide string]",
#     'department': "What are the departments he/she work with for the particular job positions that are present in the resume? Answer in points [Only provide string]",
#     'postDesignation': "What are the post designations for the particular job positions present in the resume? Answer in points and keep it precise. [Only provide string]" ,
#     'periodOfEmploymentFrom': "What are the starting tenure of work employment for the particular job positions in the resume? Answer in points and keep it precise. [Only provide integer]",
#     'periodOfEmploymentTo': "What are the ending tenure of work employment for the particular job positions in the resume? Answer in points and keep it precise [Only provide single integer]",
#     'grossSalary': "What is the gross salary for the particular job positions in the resume? [Answer in 'Rs/$' + integer value]",
#     'responsibilities': "What are the job responsibilities for the particular job positions in the resume? Answer in points and keep it precise [Only provide string]",
#     'Skills_id': "Randomly Create an skills ID as primary key for my dataset, provide a unique value for this resume? [Only provide integer]",
#     'skills': "What are the skills+frameworks+soft skills present in the resume? Answer in points [Only provide string separated by ',']",
#     'selfRating': "How does the person is doing self rating, and extract from where it is present in the resume? [Only provide 2 line string]",
#     'whyRate': "Why does the person is doing self rating, and extract from where it is present in the resume? [Only provide 2 line string]",
#     'AwardsData_id': "Randomly Create an skills ID as primary key for my dataset, provide a unique value for this resume? [Only provide single integer]",
#     'nameOfAawardsAchievements': "What are the name of awards/achievements+certificates+academic achievements  present in the resume? Answer in points and keep it precise [Only provide string]",
#     'institutionsOrganization': "What are the institution/organization that provided the awards/ certificates to the person present in the resume? Answer in points and keep it precise [Only provide string]",
#     'institutionsOrganizationName': "What are the institution/organization names that provided the awards/ certificates to the person present in the resume? Answer in points and keep it precise [Only provide string]",
#     'detailsofAwards/Cerifications': "What are the details of awards/certifications present in the resume? [Only provide string in 2 lines]",
#     'dateOfAwards': "What are the date of awards obtained present in the resume? Answer in points and keep it precise [Only provide integer]",
#     'awardsTitle': "What are the awards title obtained present in the resume? Answer in points and keep it precise [Only provide string separated by ',']",
#     'urlAwards': "What are the URL of awards present in the resume?  Answer in points and keep it precise [Only provide link]",
#     'typesOfAchievment': "What are the types of awards/achievement (Under which category they belong ex- Techical, Marketting etc.) present in the resume? Answer in points and keep it precise [Only provide string]"
# }

In [ ]:
# # For querying

# query_text = "What is the first name present in the resume?  [Only provide single string] ?"
# query_embedding = generate_embedding(query_text)
# top_indices = search_faiss_index(vecstore, query_embedding)
# print("Top similar documents indices:", top_indices)
# # After you perform the search
# top_text_chunks = get_text_for_indices(top_indices, text_chunks)
# print("Top similar documents text chunks:")
# for chunk in top_text_chunks:

#   print("Chunk --->",chunk)

In [ ]:
# min_index = min(top_indices)
# min_chunk = text_chunks[min_index]
# print(min_chunk)

In [4]:
prompt = {
    "fname": "What is the first name present in the resume?  [Only provide single string]",
    "lastName": "What is the last name present in the resume? [Only provide single string]",
    "dob": "What is the date of birth present in the resume? [Only provide single string]",
    "age": "What is the age (Current year - Date of Birth year) present in the resume? [Only provide single integer]",
    "email": "What is the email present in the resume? [Only provide single string]",
    "address": "What is the address present in the resume? [Only provide string]",
    "mobile": "What is the mobile number present in the resume? [Only provide single long integer]",
    "LinkedIn": "What is the LinkedIn profile present in the resume? [Only provide single link]",
    "orcidId": "What is the ORCID ID for the research papers present in the resume? [Only provide single id]",
    "totalIndustryExperience": "What is the total industry years of experience present in the resume. [Sum of all the years of experience and provide integer value+ 'years'] ",
    "summaryDetails": "What is the summary present in the resume? [Only provide 2 lines string]",
    "degree": "What are the degrees(10th Degree, 12th Degree, Undergrad Degree, Postgrad Degree) present in the resume? Answer in points and keep it precise. [Only provide Degree names]" ,
    "course": "What are the courses taken in the degrees present in the resume? Answer in points and keep it precise. [Only provide string separated by ',']",
    "marks": "What are the CGPA (Undergrad Degree, Postgrad Degree) present in the resume? Answer in points and keep it precise. [Only provide integer]",
    "yearOfPass": "What are the years of passing (for 10th Degree, 12th Degree, Undergrad Degree, Postgrad Degree) present in the resume? Answer in points and keep it precise. [Only provide integer]",
    "boardsName": "What are the board's names (10th Degree, 12th Degree) present in the resume? Answer in points and keep it precise. [Only provide string]",
    "percentage": "What are the percentages (10th Degree, 12th Degree) present in the resume? Answer in points  [provide integer value + '% in 10th and % in 12th'] ",
    'nameOfEmployer': "What are the names of the employers present in the resume for the particular job positions?  Answer in points and keep it precise. [Only provide string]",
    'department': "What are the departments he/she work with for the particular job positions that are present in the resume? Answer in points [Only provide string]",
    'postDesignation': "What are the post designations for the particular job positions present in the resume? Answer in points and keep it precise. [Only provide string]" ,
    'periodOfEmploymentFrom': "What are the starting tenure of work employment for the particular job positions in the resume? Answer in points and keep it precise. [Only provide integer]",
    'periodOfEmploymentTo': "What are the ending tenure of work employment for the particular job positions in the resume? Answer in points and keep it precise [Only provide single integer]",
    'grossSalary': "What is the gross salary for the particular job positions in the resume? [Answer in 'Rs/$' + integer value]",
    'responsibilities': "What are the job responsibilities for the particular job positions in the resume? Answer in points and keep it precise [Only provide string]",
    'skills': "What are the skills+frameworks+soft skills present in the resume? Answer in points [Only provide string separated by ',']",
    'selfRating': "How does the person is doing self rating, and extract from where it is present in the resume? [Only provide 2 line string]",
    'whyRate': "Why does the person is doing self rating, and extract from where it is present in the resume? [Only provide 2 line string]",
    # 'nameOfAawardsAchievements': "What are the name of awards/achievements+certificates+academic achievements  present in the resume? Answer in points and keep it precise [Only provide string]",
    # 'institutionsOrganization': "What are the institution/organization that provided the awards/ certificates to the person present in the resume? Answer in points and keep it precise [Only provide string]",
    # 'institutionsOrganizationName': "What are the institution/organization names that provided the awards/ certificates to the person present in the resume? Answer in points and keep it precise [Only provide string]",
    # 'detailsofAwards/Cerifications': "What are the details of awards/certifications present in the resume? [Only provide string in 2 lines]",
    # 'dateOfAwards': "What are the date of awards obtained present in the resume? Answer in points and keep it precise [Only provide integer]",
    # 'awardsTitle': "What are the awards title obtained present in the resume? Answer in points and keep it precise [Only provide string separated by ',']",
    # 'urlAwards': "What are the URL of awards present in the resume?  Answer in points and keep it precise [Only provide link]",
    # 'typesOfAchievment': "What are the types of awards/achievement (Under which category they belong ex- Techical, Marketting etc.) present in the resume? Answer in points and keep it precise [Only provide string]"
    'nameOfAwardsAchievements': "What are the name of awards/achievements+certificates+academic achievements  present in the resume?",
    'institutionsOrganization': "What are the institution that provided the awards/ certificates to the person present in the resume?",
    'institutionsOrganizationName': "What are the institution/organization names that provided the awards/ certificates to the person present in the resume?",
    'detailsofAwards/Cerifications': "What are the details of awards/certifications present in the resume?",
    'dateOfAwards': "What are the date of awards obtained present in the resume?",
    'awardsTitle': "What are the awards title obtained present in the resume?",
    'urlAwards': "What are the URL of awards present in the resume?",
    'typesOfAchievment': "What are the types of awards/achievement"
}

In [5]:
output_dict = {}

for key, prompt_text in prompt.items():
    query_text = prompt_text
    query_embedding = generate_embedding(query_text)
    top_indices = search_faiss_index(vecstore, query_embedding)
    top_text_chunks = get_text_for_indices(top_indices, text_chunks)
    output_dict[key] = top_text_chunks

with open("output.txt", "w", encoding="utf-8") as f:
    for key, chunks in output_dict.items():
        f.write(f"{key} : {chunks}\n")


In [6]:
import json

with open("output_extracted.json", "w") as f:
    json.dump(output_dict, f)

# Load the data from the JSON file
with open("output_extracted.json", "r") as f:
    data = json.load(f)

# Clean up the text chunks
cleaned_data = {}
for key, chunks in data.items():
    cleaned_chunks = []
    for chunk in chunks:
        # Replace '\n' with actual new lines
        cleaned_chunk = chunk.replace("\n", "\n")
        # Remove any unwanted characters
        cleaned_chunk = cleaned_chunk.strip()
        # Append the cleaned chunk to the list
        cleaned_chunks.append(cleaned_chunk)
    # Update the cleaned data dictionary
    cleaned_data[key] = cleaned_chunks

# Write the cleaned data to a new JSON file
with open("output_extracted.json", "w") as f:
    json.dump(cleaned_data, f, indent=4)


In [ ]:
# results = {}

# # Iterate through each prompt in the dictionary
# for key, prompt_text in prompt.items():
#     query_text = prompt_text
#     query_embedding = generate_embedding(query_text)
#     top_indices = search_faiss_index(vecstore, query_embedding)
#     min_index = min(top_indices)
#     min_chunk = text_chunks[min_index]
#     results[prompt_text] = min_chunk


In [ ]:
# # Write the results to a JSON file
# with open("new_output_extracted.json", "w") as json_file:
#     json.dump(results, json_file, indent=4)


In [7]:
import json

with open("output_extracted.json", "r") as f:
    extracted_data = json.load(f)

mapped_data = {}

for key, value in prompt.items():
    mapped_data[value] = extracted_data.get(key, [])

print(mapped_data)


{'What is the first name present in the resume?  [Only provide single string]': ['Dikesh Ray  \nMobile: 7827137507  \nEmail: Dikesh.ray@gmail.com  \n \nSubject: Cover Letter – Software Developer  \n \nDear Sir/Madam,  \n \nI am MCA degree holder having 5.4 years of experience in analyzing, designing and developing \nweb applications. At present, working with BLS international Services ltd. My primary involvement \nis in developing', '● Having good knowledge of HTML, CSS, BOOTSTRAP v5 JavaScript, JQuery , JSON and \nAjax, Angular js 4 & 5, REACT  js, \n● Experience with E-commerce  site. \n● Good team player and also have ability to work independently in a time sensitive \nenvironment.  \n \nPlease find enclosed copy of my CV for your consideration. Feel free to ask if any further \ninformation needed', 'a Software Developer from 05th \nJune 2017 to till  date.  \n● Working with BRAWN SOFTECH . as a Software Developer from 20th September 2016 \nto 31th March 2017.  \n● Working with HYBE

In [8]:
with open('check.txt', 'w', encoding="utf-8") as f:
    f.write("{\n")
    for question, answers in mapped_data.items():
        f.write("    '" + question + "': [\n")
        for answer in answers:
            f.write("        '" + answer.replace('\n', '\\n') + "',\n")
        f.write("    ],\n")
    f.write("}\n")

In [9]:
import openai
import time
openai.api_key = "API_KEY"  # Replace API_KEY with your actual API key
model_name = "gpt-3.5-turbo-16k-0613"

conversation = []
for question, answers in mapped_data.items():
    response = openai.ChatCompletion.create(
      model=model_name,
      messages=[
        {"role": "system", "content": f"You are provided with a dictinary {mapped_data}, extract information based on the {question} from the {answers}. If unable to extract info, (REMEMBER) DIRECTLY WRITE 'No information available' as output. Act as  an expert data analyst, with full knowledge of data extraction from the RESUME "},

        {"role": "user", "content": f"Q: {question}\nA:"}
      ],
      temperature=0.6,
      max_tokens=200,
      stop=["\n"]
  )

    result = response.choices[0].message.content.strip()
    time.sleep(0.5)
    conversation.extend([{"role": "system", "content": f"Q: {question}"}, {"role": "user", "content": result}])
    print(f"Question: {question}")
    print(f"Answer: {result}")


Question: What is the first name present in the resume?  [Only provide single string]
Answer: Dikesh
Question: What is the last name present in the resume? [Only provide single string]
Answer: Ray
Question: What is the date of birth present in the resume? [Only provide single string]
Answer: 04-feb-1990
Question: What is the age (Current year - Date of Birth year) present in the resume? [Only provide single integer]
Answer: No information available
Question: What is the email present in the resume? [Only provide single string]
Answer: Dikesh.ray@gmail.com
Question: What is the address present in the resume? [Only provide string]
Answer: E-676, street No -10, W est vinod nagar, New Delhi(110092)
Question: What is the mobile number present in the resume? [Only provide single long integer]
Answer: 7827137507
Question: What is the LinkedIn profile present in the resume? [Only provide single link]
Answer: No information available
Question: What is the ORCID ID for the research papers presen

RateLimitError: Rate limit reached for gpt-3.5-turbo-16k-0613 in organization org-GXHqOeNUlPqj3ZxrRnkX353H on tokens per min (TPM): Limit 180000, Used 177175, Requested 14147. Please try again in 3.774s. Visit https://platform.openai.com/account/rate-limits to learn more.

RateLimitError: This error is coming after the for loop is running, after giving some outputs

RateLimitError                            Traceback (most recent call last)
<ipython-input-9-fd7054751d16> in <cell line: 7>()
      6 conversation = []
      7 for question, answers in mapped_data.items():
----> 8     response = openai.ChatCompletion.create(
      9       model=model_name,
     10       messages=[

/usr/local/lib/python3.10/dist-packages/openai/api_resources/chat_completion.py in create(cls, *args, **kwargs)
     23         while True:
     24             try:
---> 25                 return super().create(*args, **kwargs)
     26             except TryAgain as e:
     27                 if timeout is not None and time.time() > start + timeout:

/usr/local/lib/python3.10/dist-packages/openai/api_resources/abstract/engine_api_resource.py in create(cls, api_key, api_base, api_type, request_id, api_version, organization, **params)
    151         )
    152 
--> 153         response, _, api_key = requestor.request(
    154             "post",
    155             url,

/usr/local/lib/python3.10/dist-packages/openai/api_requestor.py in request(self, method, url, params, headers, files, stream, request_id, request_timeout)
    296             request_timeout=request_timeout,
...
--> 765             raise self.handle_error_response(
    766                 rbody, rcode, resp.data, rheaders, stream_error=stream_error
    767             )

RateLimitError: Rate limit reached for gpt-3.5-turbo-16k-0613 in organization org-GXHqOeNUlPqj3ZxrRnkX353H on tokens per min (TPM): Limit 180000, Used 177175, Requested 14147. Please try again in 3.774s. Visit https://platform.openai.com/account/rate-limits to learn more.
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...